In [1]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.processors import ByteLevel as ByteLevelProcessor
from tokenizers import decoders

# Load streaming dataset
ds = load_dataset(
    "HuggingFaceTB/smollm-corpus", "cosmopedia-v2", split="train", streaming=True
)

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

In [2]:
# Create a text iterator (adjust column name if needed)
def batch_iterator(dataset, batch_size=1000):
    batch = []
    for sample in dataset:
        batch.append(sample["text"])  # Check your column name
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch


# Initialize tokenizer
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.pre_tokenizer = ByteLevel(add_prefix_space=False)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.post_processor = ByteLevelProcessor(trim_offsets=False)

# Setup trainer
trainer = BpeTrainer(
    vocab_size=32000,
    special_tokens=[
        "<unk>",
        "<s>",  # bos
        "</s>",  # eos
        "<pad>",
    ],
    min_frequency=2,  # skip ultra-rare tokens
    show_progress=True,
)

subset_ds = ds.take(2_000_000)
tokenizer.train_from_iterator(batch_iterator(subset_ds), trainer=trainer)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: cf900ccf-407d-49d4-bd96-a4679d36c0fc)')' thrown while requesting GET https://huggingface.co/datasets/HuggingFaceTB/smollm-corpus/resolve/3ba9d605774198c5868892d7a8deda78031a781f/cosmopedia-v2/train-00000-of-00104.parquet
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 6641a4d0-cd6b-4aad-84e5-e933ef17dc59)')' thrown while requesting GET https://huggingface.co/datasets/HuggingFaceTB/smollm-corpus/resolve/3ba9d605774198c5868892d7a8deda78031a781f/cosmopedia-v2/train-00002-of-00104.parquet
Retrying in 1s [Retry 1/5].


In [ ]:
# Save
tokenizer.save("../tokenizer/bpe_tokenizer.json")

In [4]:
# Test it
print(tokenizer.encode("Hello, world!").tokens)

['Hello', ',', 'Ġworld', '!']
